In [416]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report

import numpy as np

tf.enable_v2_behavior()

In [417]:
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

In [418]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [419]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [420]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.summary()

models = [model1]

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_35 (Flatten)         (None, 4732)              0         
_________________________________________________________________
dense_70 (Dense)             (None, 128)               605824    
_________________________________________________________________
dropout_35 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_71 (Dense)             (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
_______________________________________________

In [421]:
def tf_l2_clip(v, b):
    norm = tf.norm(v)
    return tf.cond(norm > b, lambda: b * (v / norm), lambda: v)
    
    
class EpsilonDeltaDPGradientDescent(tf.keras.optimizers.Optimizer):
    def __init__(self, epsilon, delta, b=3.0, learning_rate=0.01, name="DPGradientDescent", **kwargs):
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", learning_rate)
        
        self.epsilon = epsilon
        self.delta = delta
        self.b = b
    
    def _create_slots(self, var_list):
        pass

    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        #clipped_grad = tf.math.reduce_mean(tf_l2_clip(grad, self.b))
        clipped_grad = tf.keras.backend.mean(tf_l2_clip(grad, self.b), axis=1)
        #clipped_grad = tf_l2_clip(grad, self.b)
        new_var_m = var - tf_gaussian_mech(clipped_grad, self.b/len(x_train), self.epsilon, self.delta) * lr_t
        #new_var_m = var - grad * lr_t
        
        new_var = new_var_m
        var.assign(new_var)

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

In [422]:
es = callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
optimizer = EpsilonDeltaDPGradientDescent(100, 1e-5, 0.1)
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=1, callbacks=[es], batch_size=64)

ValueError: in user code:

    /Users/goatgoose/Library/Python/3.8/lib/python/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-421-2d165fb8780c>:24 _resource_apply_dense  *
        clipped_grad = tf.keras.backend.mean(tf_l2_clip(grad, self.b), axis=1)
    /Users/goatgoose/Library/Python/3.8/lib/python/site-packages/tensorflow/python/util/dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    /Users/goatgoose/Library/Python/3.8/lib/python/site-packages/tensorflow/python/keras/backend.py:2263 mean
        return math_ops.reduce_mean(x, axis, keepdims)
    /Users/goatgoose/Library/Python/3.8/lib/python/site-packages/tensorflow/python/util/dispatch.py:201 wrapper
        return target(*args, **kwargs)
    /Users/goatgoose/Library/Python/3.8/lib/python/site-packages/tensorflow/python/ops/math_ops.py:2309 reduce_mean
        gen_math_ops.mean(
    /Users/goatgoose/Library/Python/3.8/

In [343]:
tf.random.normal([4], 0, 1, tf.float32)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.56066424,  1.3532932 ,  1.0510182 , -0.44213298], dtype=float32)>

In [344]:
t = tf.constant([1.0, 1.0, 1.0, 1.0])
t

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>

In [345]:
t_g = tf_gaussian_mech(t, 3.0/len(x_train), 0.1, 1e-5)
t_g

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1.0018165 , 0.9987338 , 0.99778914, 1.0040578 ], dtype=float32)>

In [346]:
norm = tf.norm(t)
norm.numpy()

2.0